# Imports

In [124]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [125]:
# 커스텀 파라미터 -> 모든 열 보이게
pd.set_option('display.max_columns', None)

# 데이터 불러오기

In [126]:
df_newb = pd.read_parquet('./data/원본/parquet/신규가입자정보.parquet', engine='pyarrow')
df_rental = pd.read_parquet('./data/원본/parquet/대여이력정보.parquet', engine='pyarrow')
df_broken = pd.read_parquet('./data/원본/parquet/고장신고내역.parquet', engine='pyarrow')

신규가입자 정보

In [127]:
df_newb

,가입일시,가입건수,year
0,202207,56393,2022
1,202208,53853,2022
2,202209,81574,2022
3,202210,62511,2022
4,202211,36600,2022
5,202212,11530,2022
6,202307,42649,2023
7,202308,41346,2023
8,202309,61111,2023
9,202310,66836,2023


대여이력정보
내용이 너무 많아서 20%만 샘플링해서 진행

In [128]:
df_rental = df_rental.sample(frac=0.2, random_state=3)
df_rental

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),생년,성별,이용자종류,대여대여소ID,반납대여소ID,자전거구분
19030019,SPB-39992,2022-11-10 23:09:05,1124,발산역 6번 출구 뒤,0,2022-11-11 00:46:01,00162,봉원고가차도 밑,0,96,14120.01,1975,F,내국인,ST-522,ST-219,None
16698934,SPB-39988,2022-10-26 17:00:50,3571,화양 APT(횡단보도 옆),0,2022-10-26 17:14:44,03569,건대병원후문,0,13,1030.44,2002,\N,내국인,ST-2310,ST-2308,None
8710569,SPB-50825,2022-09-06 19:17:39,1158,가양역 8번출구,0,2022-09-06 19:24:06,01182,KBS 스포츠월드,0,6,887.81,1991,\N,내국인,ST-1249,ST-1519,None
1929187,SPB-52800,2022-07-15 06:54:34,720,서울강월초등학교 앞,0,2022-07-15 07:36:49,01102,방화사거리 마을버스 버스정류장,0,42,7356.28,1989,F,내국인,ST-432,ST-508,None
20404772,SPB-53662,2022-11-21 12:56:34,1440,면목2동 사거리,0,2022-11-21 13:50:41,01440,면목2동 사거리,0,54,0.00,1989,F,내국인,ST-942,ST-942,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23659877,SPB-63489,2023-07-03 11:28:50,1195,코오롱One&Only타워,0,2023-07-03 11:34:48,02715,마곡나루역 2번 출구,0,5,900.00,1998,\N,내국인,ST-1712,ST-2031,일반자전거
7463799,SPB-44935,2022-08-27 17:40:58,1246,문정 법조단지10,0,2022-08-27 17:43:28,01246,문정 법조단지10,0,2,0.00,1998,\N,내국인,ST-906,ST-906,None
20011494,SPB-65752,2022-11-18 15:11:09,755,목동1단지아파트 상가 앞,0,2022-11-18 15:20:01,00769,CBS방송국 앞,0,8,1450.00,1972,\N,내국인,ST-1496,ST-1995,None
25300035,SPB-65234,2023-07-16 22:09:31,3534,건대입구역 5번출구 뒤,0,2023-07-16 22:23:25,03575,자양사거리(서원빌딩),0,13,1180.00,1989,F,내국인,ST-1661,ST-2314,일반자전거


고장신고내역

In [129]:
df_broken

,자전거번호,등록일시,고장구분
0,SPB-30229,2022-07-01 00:44:33,페달
1,SPB-40134,2022-07-01 00:57:59,페달
2,SPB-40789,2022-07-01 01:08:25,안장
3,SPB-50932,2022-07-01 01:09:33,체인
4,SPB-41175,2022-07-01 01:13:06,안장
...,...,...,...
194724,SPB-60450,2023-12-31 23:00:11,체인
194725,SPB-30500,2023-12-31 23:29:38,안장
194726,SPB-30500,2023-12-31 23:29:38,기타
194727,SPB-30224,2023-12-31 23:49:38,체인


# EDA

**확인해야 할 것들:**

1. 데이터타입 -> 날짜 데이터 datetime / period으로 변환

2. 중복값 확인 및 제거

3. 1차 변수 선별
- 열 삭제: 자전거구분, 성별, 대여거치대, 반납거치대, 대여 대여소번호, 대여 대여소명, 반납일시, 반납대여소번호, 반납대여소명 (반납대여소ID와 결측치 수도 같아서 대체 가능)
- 행 삭제: 이용자종류 -> 외국인 삭제


4. 결측치
- \N 값 결측치로 변환
- 결측치 확인
- 행 삭제: 반납대여소ID (정상적인 반납이 아니기에, 오류로 분류), 생년 (나이대가 14세부터 64세가 필요해서, 데이터량이 많아서)

5. 이상치
- 이용시간(분): 5분 이상, 60분 이하
- 이용거리(M): 1km 이상, 15km 이하

6. 파생변수:
- 생년: 나이(만)으로 14~64세로 줄인 후 생성

7. 2차 변수 선별
- 열 삭제: 생년, 이용자종류

## 날짜 데이터 변환

날짜 데이터 변환
1. 신규가입자정보 날짜 들여다보기 쉽게 `가입일시` period로 변환, year 컬럼 제거 -> 나중에 `.year` 또는 `.month` 로 년월 호출 가능
2. 대여이력정보 `대여일시` datetime으로 변환
3. 고장신고내역 `등록일시` datetime으로 변환

In [130]:
# 가입일시 period 데이터타입 변환
df_newb['가입일시'] = (
    df_newb['가입일시']
    .astype('str')
    .apply(lambda x: x[:4] + '-' + x[4:])
    .apply(lambda x: pd.to_datetime(x)).dt.to_period('M')
)
# year 컬럼 제거
df_newb.drop(columns=['year'], inplace=True)

In [131]:
# 대여이력정보 대여일시 datetime 데이터타입 변환
df_rental['대여일시'] = pd.to_datetime(df_rental['대여일시'])
# 고장신고내역 등록일시 datetime 데이터타입 변환
df_broken['등록일시'] = pd.to_datetime(df_broken['등록일시'])

## 중복값 확인 및 제거

In [132]:
df_newb.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df_rental.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df_broken.drop_duplicates(keep='first', inplace=True, ignore_index=True)

## 1차 변수 선별 (대여이력정보)

- 열 삭제: 자전거구분, 성별, 대여거치대, 반납거치대, 반납대여소번호, 반납대여소명 (반납대여소ID와 결측치 수도 같아서 대체 가능)

In [133]:
df_rental.drop(columns=['자전거구분', '성별', '대여거치대', '반납일시', '반납거치대', '대여 대여소번호', '대여 대여소명', '반납대여소번호', '반납대여소명'], inplace=True)

- 행 삭제: 이용자종류 -> 외국인 삭제

In [134]:
user_foreigners = df_rental.loc[df_rental['이용자종류'] == '외국인'].index
df_rental.drop(index=user_foreigners, inplace=True)

## 결측치 확인 및 처리

- \N 값 결측치로 변환

In [135]:
df_rental.replace('\\N', np.nan, inplace=True)

- 결측치 확인

In [136]:
print(f"신규가입자 결측치:\n    NaN: {df_newb.isna().sum().sum()}")
print(f"대여이력정보 결측치:\n    NaN: {df_rental.isna().sum().sum()}")
print(f"고장신고내역 결측치:\n    NaN: {df_broken.isna().sum().sum()}")

rental_na = df_rental.isna().sum()
rental_na_ratio = (rental_na / len(df_rental))
df_rental_na = pd.DataFrame({
    'NA (count)': rental_na,
    'NA (ratio)': rental_na_ratio
})
df_rental_na = df_rental_na[df_rental_na['NA (count)'] > 0]
df_rental_na

신규가입자 결측치:
    NaN: 0
대여이력정보 결측치:
    NaN: 765803
고장신고내역 결측치:
    NaN: 0


,NA (count),NA (ratio)
생년,726509,0.076802
반납대여소ID,39294,0.004154


- 행 삭제: 반납대여소ID (정상적인 반납이 아니기에, 오류로 분류), 생년 (나이대가 14세부터 64세가 필요해서, 데이터량이 많아서)

In [137]:
df_rental.dropna(subset=['반납대여소ID', '생년'], how='any', axis=0, inplace=True)

## 이상치 확인 및 처리

- 이용시간(분): 5분 이상, 60분 이하

In [138]:
less_than_5min = df_rental['이용시간(분)'] <= 5
greater_than_60min = df_rental['이용시간(분)'] >= 60
outlier_time = df_rental.loc[less_than_5min | greater_than_60min].index
df_rental.drop(index=outlier_time, inplace=True)

- 이용거리(M): 1km 이상, 15km 이하

In [139]:
less_than_1km = df_rental['이용거리(M)'] <= 1000
greater_than_10km = df_rental['이용거리(M)'] >= 15000
outlier_distance = df_rental.loc[less_than_1km | greater_than_10km].index
df_rental.drop(index=outlier_distance, inplace=True)

## 파생변수

- 생년: 나이(만)으로 14~64세로 줄인 후 생성

In [140]:
# 나이(만) 컬럼 생성
age = df_rental['대여일시'].dt.year - df_rental['생년'].astype('int')
df_rental['나이(만)'] = age

# 14세 미만 64세 초과 제거
not_needed_ages = df_rental.loc[(df_rental['나이(만)'] < 14) | (df_rental['나이(만)'] > 64)].index
df_rental.drop(index=not_needed_ages, inplace=True)

## 2차 변수 선별

- 열 삭제: 반납일시, 생년, 이용자종류

In [141]:
df_rental.drop(columns=['생년', '이용자종류'], inplace=True)

In [142]:
df_rental

,자전거번호,대여일시,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,나이(만)
1,SPB-39988,2022-10-26 17:00:50,13,1030.44,ST-2310,ST-2308,20
3,SPB-52800,2022-07-15 06:54:34,42,7356.28,ST-432,ST-508,33
5,SPB-50317,2022-11-23 07:57:05,16,2113.73,ST-1990,ST-1973,24
8,SPB-32747,2022-07-01 19:17:47,14,1341.94,ST-81,ST-90,27
9,SPB-61404,2022-10-07 07:43:42,11,1255.32,ST-2127,ST-1103,59
...,...,...,...,...,...,...,...
9475048,SPB-33482,2022-09-07 18:43:04,21,2502.36,ST-400,ST-3046,31
9475049,SPB-36710,2023-12-07 18:10:32,13,1392.62,ST-1753,ST-3219,29
9475052,SPB-65752,2022-11-18 15:11:09,8,1450.00,ST-1496,ST-1995,50
9475053,SPB-65234,2023-07-16 22:09:31,13,1180.00,ST-1661,ST-2314,34
